In [1]:
import pandas as pd
df0 = pd.read_csv('../assets/data/master.txt')

print(df0.shape)
df0.head()

(10202, 19)


,player,date,team,home,opp,game,week,day,completions,passatt,passyards,passtds,ints,sacks,sackyards,rushatt,rushyards,rushtds,fumbles
0,Matt Flynn,2013-12-22,GNB,NaN,PIT,15,16,Sun,21,39,232,1,1,3.0,13.0,2,8,0,1
1,Nick Foles,2013-12-22,PHI,NaN,CHI,15,16,Sun,21,25,230,2,0,2.0,5.0,2,17,0,0
2,Mike Glennon,2013-12-22,TAM,@,STL,15,16,Sun,16,26,158,0,0,7.0,47.0,2,3,0,1
3,Chad Henne,2013-12-22,JAX,NaN,TEN,15,16,Sun,24,34,237,2,1,2.0,11.0,1,1,0,0
4,Josh Johnson,2013-12-22,CIN,NaN,MIN,15,16,Sun,0,0,0,0,0,0.0,0.0,4,3,0,0


In [2]:
import numpy as np

def sanitize(df):
    df = df.copy()
    
    # "home" is NaN when true, "@" when false;
    # make this explicit
    df['home'] = df['home'].fillna(1)
    df['home'] = df['home'].replace('@', 0)
    
    # exclude postseason games
    df = df[df['game'] < 17]
    
    # drop "games" with no pass attempts, sacks, or rushes
    haspass = df['passatt'] > 0
    hassack = df['sacks'] > 0
    hasrush = df['rushatt'] > 0
    df = df[haspass | hassack | hasrush]
    
    # remove players with fewer than 100 games
    vc = (df['player'].value_counts() >= 100).to_dict()
    enough = [p for p in vc if vc[p]]
    df = df[df['player'].isin(enough)]
    
    # fill missing values
    df['sacks'] = df['sacks'].fillna(0)
    df['sackyards'] = df['sackyards'].fillna(0)
    
    # some player names have a trailing *
    df['player'] = df['player'].str.replace(r'\*$', '')
    
    # engineer a season-year feature
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df['season'] = df['date'].apply(lambda x: x.year if x.month > 6 else x.year - 1)
    
    # drop some columns
    ignore = ['date', 'team', 'home', 'opp', 'game', 'week', 'day']
    df = df[df.columns.drop(ignore)]
    
    return df;

df1 = sanitize(df0)
print(df1.shape)
df1.head()

(3729, 13)


,player,completions,passatt,passyards,passtds,ints,sacks,sackyards,rushatt,rushyards,rushtds,fumbles,season
7,Eli Manning,23,42,256,1,1,2.0,18.0,2,-2,0,1,2013
8,Peyton Manning,32,51,400,4,0,1.0,3.0,0,0,0,0,2013
13,Cam Newton,13,22,181,1,1,4.0,40.0,4,6,0,0,2013
14,Carson Palmer,13,25,178,1,4,2.0,10.0,5,-3,0,0,2013
15,Philip Rivers,19,29,201,1,1,1.0,5.0,4,-3,0,1,2013


In [3]:
df1.to_csv('../assets/data/sanitized.txt', index=False)

In [4]:
df1['player'].value_counts(normalize=True)

Drew Brees            0.064629
Eli Manning           0.062483
Tom Brady             0.061947
Philip Rivers         0.059802
Ben Roethlisberger    0.058461
Matt Ryan             0.049075
Carson Palmer         0.048807
Aaron Rodgers         0.046929
Joe Flacco            0.045857
Peyton Manning        0.045589
Alex Smith            0.044516
Jay Cutler            0.041030
Matthew Stafford      0.039957
Ryan Fitzpatrick      0.039689
Tony Romo             0.035398
Matt Hasselbeck       0.035398
Andy Dalton           0.034326
Cam Newton            0.033521
Russell Wilson        0.032717
Matt Schaub           0.032180
Michael Vick          0.030571
Brett Favre           0.029230
Matt Cassel           0.027890
Name: player, dtype: float64